In [1]:
import sys
import itertools
import logging
from math import sqrt
from operator import add
from os.path import join, isfile, dirname

from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.sql import SparkSession

import json
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes

# from pycorenlp import StanfordCoreNLP
# nlp = StanfordCoreNLP('http://localhost:9000')

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
print("Running Spark Version %s" % (spark.version))

#### load business data ###
# path_business ="yizhan/Desktop/cs181/dataset/buiness.json"
path_business = "dataset/business.json"
df_business_raw = spark.read.json(path_business)
# df_business_raw.first()

businessDF = df_business_raw.select(df_business_raw["business_id"], df_business_raw["name"], df_business_raw["categories"])
print(businessDF.first())
businessDF.printSchema()
businessDF.createOrReplaceTempView("business")

# path_review="yizhan/Desktop/CS181/yelp dataset/review.json"
path_review="dataset/review.json"
df_review_raw = spark.read.json(path_review)
# df_review_raw.first()

reviewDF= df_review_raw.select(df_review_raw["review_id"], df_review_raw["user_id"], df_review_raw["business_id"], df_review_raw["text"])
print(reviewDF.first())
reviewDF.printSchema()
reviewDF.createOrReplaceTempView("review")

# join reviews with busines on businnes_id 
merged_review = spark.sql("SELECT review.business_id, user_id, text, categories, name FROM review join business on review.business_id == business.business_id")

Running Spark Version 2.2.0
Row(business_id='YDf95gJZaq05wvo7hTQbbQ', name='Richmond Town Square', categories=['Shopping', 'Shopping Centers'])
root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)

Row(review_id='VfBHSwC5Vz_pbFluy07i9Q', user_id='cjpdDjZyprfyDG3RlkVG3w', business_id='uYHaNptLzDLoV_JZ_MuzUA', text='My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comforta

In [3]:
 merged_review.head()

Row(business_id='--9e1ONYQuAa-CB_Rrw7Tw', user_id='4LxKRRIikhr65GfPDW626w', text="What can I say.. Wowzers! Probably one of the best steak houses I've been too. Service was absolutely flawless and dinner was excellent . Ordered seafood tower, wedge, wagyu filet, chateaubriand, bacon grits and sautéed  mushrooms Will definitely be back!", categories=['Steakhouses', 'Cajun/Creole', 'Restaurants'], name='Delmonico Steakhouse')

In [129]:
sample = merged_review.rdd.take(10000)
# sample = merged_review.rdd

In [130]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover
from random import randint

<class 'list'>


In [220]:
# duplicate data with multiple features
list = []
label_to_num_dict = {}
num_to_label_dict = {}
index = 0 
# iteratorlist = sample.collect()
# for line in iteratorlist:
for line in sample:
    labels = line["categories"]
    # print(labels)
    text = line["text"]
    # print(text)
    
    append = False
    for label in labels:
        if label == 'Restaurants' or label == 'Food' :
            append = True
    
    if append == True:       
        label_index_to_choose = randint(0, len(labels)-1)
        label = labels[label_index_to_choose]
        while label == 'Restaurants'or label == 'Food':
            label_index_to_choose = randint(0, len(labels)-1)
            label = labels[label_index_to_choose]
            
        if label not in label_to_num_dict:
            # print(index, " represents ",label)
            label_to_num_dict[label] = index
            num_to_label_dict[index] = label 
            index += 1  
                
        num_label = label_to_num_dict[label] # covert the category label to numerical value
        if ( num_label < 10 ):
            print(label, labels, text)
            list.append([num_label,text])
        
        '''
        for label in labels:
            if label == 'Restaurants'or label == 'Food':
                continue
            if label not in label_to_num_dict:
                # print(index, " represents ",label)
                label_to_num_dict[label] = index
                num_to_label_dict[index] = label 
                index += 1  
            # covert the category label to numerical value
            num_label = label_to_num_dict[label] 
            list.append([num_label,text])
            break
        '''

# print(list)

Cajun/Creole ['Steakhouses', 'Cajun/Creole', 'Restaurants'] What can I say.. Wowzers! Probably one of the best steak houses I've been too. Service was absolutely flawless and dinner was excellent . Ordered seafood tower, wedge, wagyu filet, chateaubriand, bacon grits and sautéed  mushrooms Will definitely be back!
Cajun/Creole ['Steakhouses', 'Cajun/Creole', 'Restaurants'] We went in with high hopes after a friend recommended Delmonico but we left just a bit disappointed.  The ambiance was really nice and we were given a great table without a reservation but the food left a little to be desired.  

My friend and I both started with lobster bisque, which was good but a little thin.  We noticed right away too that she had 2 tiny little pieces of lobster in hers, but I had almost an entire small claw.  I was happy until I realized it was tough and rubbery.  I actually had to use a knife and fork to cut it into bite sized pieces.  My husband ordered the wedge salad as a starter and it was 

Steakhouses ['Steakhouses', 'Cajun/Creole', 'Restaurants'] On our stay at the Venetian, we decided to try this steak house one night.
I had the ribeye and my wife went for the filet mignon. Both steaks were absolutely delicious and cooked to perfection. The server was extremely knowledgeable about the menu .
For the desert we opted to share the bread pudding,and I have to say it is one of the best I have ever eaten.
Very good experience overall!
Steakhouses ['Steakhouses', 'Cajun/Creole', 'Restaurants'] My first review on yelp. Only reason I'm doing this review is because of Delmonico's Steakhouse at The Venetian. We were tired of the average buffets and fast food and we really wanted to have a fancy night. So we looked up on yelp for a nice fancy restaurant. We decided on Delmonico's and called in to make reservations for two at 530. Came 530, we were still at the pool, so we called in and pushed the reservation to 630. Staff accommodated us, which was surprising. We came to restauran

I remember Delmonico as a solid 4 star steak spot, and wonder what has happened. It's very sad to see this happen to what used to be one of my favorites.
Cajun/Creole ['Steakhouses', 'Cajun/Creole', 'Restaurants'] Food was delicious!! I had the salmon with the  potato hash, my girlfriend had the chicken and waffles. For sides we had the cheesy grits and asparagus. LOVED the popovers.

The service was weird... our waiter was knowledgeable and professional but seemed preoccupied, always looking around while talking to us. There was a group at the table next to us and he seemed much more attentive to them. 

Another waiter came over at the end of our dinner and we asked what spice was used in the waffle- he said he didn't know. I then asked where The Dorsey was in proximity to the restaurant, he said- again, I don't know........ he didn't even attempt to answer our questions. Great food, less than adequate service.
Steakhouses ['Steakhouses', 'Cajun/Creole', 'Restaurants'] This is the thi

I'd definitely come here again, but after I try other steakhouses in Vegas!
Cajun/Creole ['Steakhouses', 'Cajun/Creole', 'Restaurants'] I have to say that my steak which was the ribeye on bone was tasty.My friends steak being the kobe steak, was dry. Not what i expected for a Kobe steak. But considering that mines was good, I give it 4 stars. Kobe steak 3.
Cajun/Creole ['Steakhouses', 'Cajun/Creole', 'Restaurants'] Two words, food and service.  The service is top notch.  The food came timely, and everyone made sure my dining experience was exceptional.  The food, excellent without a doubt.  I had the fillet and it was very good.  Without a doubt I will definitely eat here again whenever in Las Vegas.  I would recommend this restaurant to anyone.   I finished with the banana cream pie, it was a very large piece of pie.
Steakhouses ['Steakhouses', 'Cajun/Creole', 'Restaurants'] We had dinner reservations to celebrate a special event. We started off with a few drinks.

Fried Soft-Shell Cr

Chinese ['Restaurants', 'Chinese'] Quick and good quality food. This has become our go to pick up spot for low key nights. :-)
Chinese ['Restaurants', 'Chinese'] Looks can be deceiving... 
When you come in, you will be surpised on how the tables are set up with white linen, formal table setting awaits you and a server that wears a black bow tie. This is grand and intimidating but honestly it feels great.

I went here for lunch with a friend and a dinner with my hubby. Both of them enjoyed their food and the concept of the place. The food portion, quality of food and service are amazing. I actually enjoyed the crispy wonton with a sweet and sour sauce for compliment. It's so addicting! 

Try this place if you are craving for Chinese food and you'll love it.
Chinese ['Restaurants', 'Chinese'] Absolutely gross. So disappointed  the orange chicken was chewy and the beef with broccoli was questionable. The wonton chips and cream cheese wonton have a weird yellow tint to them don't know if t

Pizza ['Italian', 'Pizza', 'Delis', 'Restaurants'] The food is good. No question about it, but the pricing is all over the board. Pizza/calzones are underpriced and pasta is super overpriced. Example:  $11 for 1 cup of pasta with 3 small slices of not toasted bread. No prices on the menu for drinks. Small fountain drink is around $2.50; a tad high.  Meatball sub was amazing. This place is not on the level of Cugino's on Maryland, but could get there with some slight changes.  Thank god they didn't call the place "Tony Boloney's".
Pizza ['Italian', 'Pizza', 'Delis', 'Restaurants'] Finally, amazing Italian food on the SouthWest side of town!! 
Ordered the Pizza Piccante - Italian sausage, spicy broccoli rabe, mozzarella 
And Angry Sicilian Sub - Prosciutto, capicola, soppressata, mortadella, mozzarella, red peppers, lettuce, tomatoes, onions, red wine vinegar and oregano.
Their home-made mozzarella is mouthwatering!
The owner Glen is super nice and friendly. He mentioned that many of the

In [214]:
print(len(list))

1687


In [228]:
for k, v in label_to_num_dict.items():
    print(k, v)

Cajun/Creole 0
Steakhouses 1
Coffee & Tea 2
Fast Food 3
Burgers 4
Chinese 5
Mexican 6
French 7
German 8
Pizza 9
Bars 10
American (New) 11
Nightlife 12
Wine Bars 13
Delis 14
Grocery 15
Bakeries 16
Caterers 17
Event Planning & Services 18
Specialty Food 19
Hot Dogs 20
Barbeque 21
Ice Cream & Frozen Yogurt 22
Shaved Ice 23
Vietnamese 24
Cafes 25
Japanese 26
Korean 27
Asian Fusion 28
Thai 29
Donuts 30
Caribbean 31
Convenience Stores 32
Drugstores 33
Shopping 34
Photography Stores & Services 35
Beauty & Spas 36
Cosmetics & Beauty Supply 37
Bagels 38
Chicken Wings 39
American (Traditional) 40
Sports Bars 41
Chocolatiers & Shops 42
Desserts 43
Comfort Food 44
Italian 45
Sandwiches 46
Breakfast & Brunch 47
Diners 48
Candy Stores 49
Hotels & Travel 50
Bed & Breakfast 51
Hotels 52
Bistros 53
Seafood 54
Salad 55
Fondue 56
Cocktail Bars 57
Vegetarian 58
Gluten-Free 59
Scottish 60
Southern 61
Health Markets 62
Organic Stores 63
Wraps 64
Soup 65
Sushi Bars 66
Portuguese 67
Indian 68
Hawaiian 69
Ethn

In [221]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# convert to rdd 
sample_with_dup = spark.sparkContext.parallelize(list)
label = sample_with_dup.map(lambda item: item[0])
text = sample_with_dup.map(lambda item: item[1])

schemaString = "label text"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

sampleDF = spark.createDataFrame(sample_with_dup, schema)
sampleDF.printSchema()

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(sampleDF)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=50)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

sampleDF.printSchema()


root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)

root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)



In [227]:
sampleDF.createOrReplaceTempView("sample")
show_count = spark.sql("SELECT DISTINCT sample.label, count(*) FROM sample GROUP BY label")
show_count.show()

+-----+--------+
|label|count(1)|
+-----+--------+
|    7|       2|
|    3|      49|
|    8|      10|
|    0|     693|
|    5|     297|
|    6|     163|
|    9|     240|
|    1|    1002|
|    4|      93|
|    2|      74|
+-----+--------+



In [216]:
# display the generated tfidf 
print("tfidf:")
count = 1
for each in rescaledData.collect():    
    print(each)
    count += 1
    if count > 1:
        break

rescaledData.select("label", "features").show()

tfidf:
Row(label='0', text="What can I say.. Wowzers! Probably one of the best steak houses I've been too. Service was absolutely flawless and dinner was excellent . Ordered seafood tower, wedge, wagyu filet, chateaubriand, bacon grits and sautéed  mushrooms Will definitely be back!", words=['what', 'can', 'i', 'say..', 'wowzers!', 'probably', 'one', 'of', 'the', 'best', 'steak', 'houses', "i've", 'been', 'too.', 'service', 'was', 'absolutely', 'flawless', 'and', 'dinner', 'was', 'excellent', '.', 'ordered', 'seafood', 'tower,', 'wedge,', 'wagyu', 'filet,', 'chateaubriand,', 'bacon', 'grits', 'and', 'sautéed', '', 'mushrooms', 'will', 'definitely', 'be', 'back!'], rawFeatures=SparseVector(50, {2: 2.0, 3: 2.0, 6: 3.0, 9: 1.0, 10: 2.0, 11: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 19: 1.0, 20: 1.0, 21: 1.0, 22: 1.0, 23: 1.0, 24: 2.0, 25: 1.0, 26: 1.0, 28: 1.0, 29: 1.0, 31: 1.0, 32: 1.0, 33: 3.0, 34: 3.0, 35: 1.0, 36: 1.0, 43: 1.0, 44: 1.0, 47: 1.0, 48: 2.0, 49: 1.0}), features=SparseVector(50, {2:

In [145]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml import linalg as ml_linalg


In [146]:
def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))

def getLabeledPoints(item):
    label = item.label
    features = as_mllib(item.features)
    return LabeledPoint(label,features)

In [229]:
data = rescaledData.rdd.map(lambda item : getLabeledPoints(item))

training, test = data.randomSplit([0.6, 0.4])

model = NaiveBayes.train(training, 1.0)

# Make prediction and test accuracy.
predictionAndLabel = test.map(lambda p: [model.predict(p.features), p.label])
accuracy = 1.0 * predictionAndLabel.filter(lambda item: item[0] == item[1]).count() / test.count()

# Save and load model
# model.save(sc, "target/tmp/myNaiveBayesModel")
# sameModel = NaiveBayesModel.load(sc, "target/tmp/myNaiveBayesModel")

In [230]:
print(accuracy)

0.4051477597712107


In [231]:
for item in predictionAndLabel.collect():
    if item[0] != item[1]:
        prediction = num_to_label_dict[item[0]]
        expected = num_to_label_dict[item[1]]
        print("prediction:", prediction, "expected label: ", expected)

prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Chinese expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Chinese expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected label:  Cajun/Creole
prediction: Steakhouses expected lab

prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Cajun/Creole expected label:  Steakhouses
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Mexican
prediction: Steakhouses expected label:  Mexican
prediction: Steakhouses expected label:  Mexican
prediction: Steakhouses expected label:  Mexican
prediction: Steakhouses expected label:  Mexican
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Steakhouses expected label:  Burgers
prediction: Ste